This script creates watersheds for the escapement monitoring locations.
This will make it easier to assign temperature and discharge monitoring
sites with salmon stocks/populations. We are also using watersheds for
some of the precipitation metrics.

Using the MERIT-Basins stream network and catchments. These were created
using a 90m DEM and TauDEM.

Reviewing all sites in the Analysis-Summary document dated 5/8/22 created by Megan and Curry.
For Kuskokwim and Canadian Yukon sites, we already have watersheds.
- Kuskokwim region - 14 populations. Keep Tuluksak in per Megan's site list and confirmation over chat.
- Canadian Yukon region - 8 pops. Keep Carmacks in per Megan's site list. We are using environmental information from the major spawning areas for each of the Canadian Yukon genetic substocks, not Brandon's watersheds, which are much too large.
- Other US pops - 6 that need watersheds.
- Create watersheds/boundaries for all Kuskokwim populations, US Yukon populations and all major spawning areas from Brown et al for the Yukon. Watersheds created using Merit-basins look better than those provided by Brendan for the Kuskokwim populations.
- Add in points to create watersheds for all major spawning populations in Yukon.

Note: Check with team on boundaries for middle and lower yukon. (Currently using points close to Yukon mouth and a point below Tanana confluence, but above Tozitna.)

Reviewed locations for US Yukon, Norton Sound sites, and major spawning areas to make sure they were in correct catchment, using shifted dataset where locations have been QAed.

1. intersect points with catchments to get catchment ID
2. create list of IDs for which we want watersheds
3. use loop to identify upstream catchments
4. merge catchments to create watersheds
5. use spatial join to get location names onto watersheds (location_short matches site spreadsheet created by Megan on google drive -- her populations are linked to major spawning areas).





In [2]:
# get list of catchment ids for escapement sites
# These include sites used for analysis that don't have watersheds already
# and major spawning populations that we will use for some precip metrics
# for the genetic population units.

import arcpy
import pandas as pd

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(4326)  #'WGS 1984'
arcpy.env.outputCoordinateSystem = sr

points = ayk_chinook_gdb + "\\chinook_sites_fromR_shiftedToMB"
cats = r"W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"
idList = []
outcats = ayk_chinook_gdb + "\\cats_esc_int"

arcpy.MakeFeatureLayer_management(points, "pointsTempLayer")
arcpy.management.SelectLayerByAttribute("pointsTempLayer", "NEW_SELECTION", "analysis_site = '1' Or analysis_site = '2' And Management_Area = 'Kuskokwim'", None)

arcpy.MakeFeatureLayer_management(cats, "tempLayer")
arcpy.SelectLayerByLocation_management("templayer", "INTERSECT", "pointsTempLayer")
arcpy.CopyFeatures_management("templayer", outcats)

fields = arcpy.ListFields("tempLayer")
for field in fields:
    print("{0}".format(field.name))
with arcpy.da.SearchCursor("tempLayer", ["COMID"]) as cursor:
    for row in cursor:
        idList.append(row[0])

print(len(idList))


FID
Shape
COMID
unitarea
49


In [3]:
# Find upstream COMIDs and merge to create watersheds in memory and put in a list


import os
import arcpy
import pandas as pd

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(4326)  #'WGS 1984'
arcpy.env.outputCoordinateSystem = sr

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
streams = r"W:\GIS\MERIT-Basins\riv_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"
cats = r"W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"

# Get list of index names for cats and add index if not already created
index_names = [i.name for i in arcpy.ListIndexes(cats)]
print(index_names)
if 'comid_index' not in index_names:
    print (f'Creating index for {cats}')
    arcpy.AddIndex_management(cats, "COMID", "comid_index")
else:
    print(f'{cats} Indexed')

#watersheds feature dataset for storing fcs
# fdat = os.path.join(ayk_chinook_gdb, 'Watersheds')
# if not arcpy.Exists(fdat):
#     arcpy.management.CreateFeatureDataset(ayk_chinook_gdb, "Watersheds", sr)
# else:
#     print(f'{fdat} exists for AYK Chinook geodatabase')


fields = arcpy.ListFields(streams)
for field in fields:
    print("{0}".format(field.name))

str_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(streams, ("COMID", "up1", "up2", "up3")))
hws_codes = [0]

#idList if doing ALL watersheds.
wtds_out = list()
c = 1
for id in idList:
    print(f"{c}. Starting watershed for: " + str(id))
    rec = [id]
    up_ids = []
    sum_rec = sum(rec)

    while(sum_rec > 0):
        up_ids.append(rec)
        rec = str_df.loc[str_df["COMID"].isin(rec), ("up1", "up2", "up3")]
        rec = pd.concat([rec['up1'], rec['up2'], rec['up3']])
        sum_rec = sum(rec)

    #up_ids is a list with more than numbers, use extend to only keep numeric comids
    newup_ids = []
    for x in up_ids:
        newup_ids.extend(x)

    tempLayer = "catsLyr"
    expression = '"COMID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
    arcpy.MakeFeatureLayer_management(cats, tempLayer)
    arcpy.management.SelectLayerByAttribute(tempLayer, "NEW_SELECTION", expression, None)
    print("Starting dissolve")
    #outdis = "memory/wtd_" + str(round(id))
    outwtd = os.path.join('in_memory', "wtd_" + str(round(id)))
    arcpy.Dissolve_management(tempLayer, outwtd)
    #watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
    arcpy.AddField_management(outwtd,"outlet_comid")
    arcpy.CalculateField_management(outwtd, "outlet_comid", id)
    print("Watershed created at:" + outwtd)
    wtds_out.append(outwtd)
    c=c+1

print(wtds_out)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')


['COMID']
Creating index for W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp
FID
Shape
COMID
lengthkm
lengthdir
sinuosity
slope
uparea
order
strmDrop_t
slope_taud
NextDownID
maxup
up1
up2
up3
up4
1. Starting watershed for: 81034341.0
Starting dissolve
Watershed created at:in_memory\wtd_81034341
2. Starting watershed for: 81007648.0
Starting dissolve
Watershed created at:in_memory\wtd_81007648
3. Starting watershed for: 81007406.0
Starting dissolve
Watershed created at:in_memory\wtd_81007406
4. Starting watershed for: 81007086.0
Starting dissolve
Watershed created at:in_memory\wtd_81007086
5. Starting watershed for: 81006916.0
Starting dissolve
Watershed created at:in_memory\wtd_81006916
6. Starting watershed for: 81010739.0
Starting dissolve
Watershed created at:in_memory\wtd_81010739
7. Starting watershed for: 81015645.0
Starting dissolve
Watershed created at:in_memory\wtd_81015645
8. Starting watershed for: 81010487.0
Starting dissolve
Watershed created at:in_me

In [4]:
# Merge all watersheds into one feature class using list from code chunk above.
import arcpy
arcpy.env.overwriteOutput = True

arcpy.env.workspace = ayk_chinook_gdb
wtd_output = "wtds_merge"
wtd_merge = arcpy.Merge_management(wtds_out, wtd_output)

print('Process complete')

Process complete


In [6]:
# Get fields from points feature class onto watersheds
# 1. spatial join between points and comid in catchments.
# 2. use join fields to merge point attributes to watersheds using comids.

import arcpy
import pandas as pd

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(4326)  #'WGS 1984'
arcpy.env.outputCoordinateSystem = sr

points = os.path.join(ayk_chinook_gdb, "chinook_sites_fromR_shiftedToMB")
out_pts = os.path.join(ayk_chinook_gdb, "chinook_sites_comid")
wtds = os.path.join(ayk_chinook_gdb, "wtds_merge")
cats = os.path.join(ayk_chinook_gdb, "cats_esc_int")

arcpy.MakeFeatureLayer_management(points, "pointsTempLayer")
arcpy.management.SelectLayerByAttribute("pointsTempLayer", "NEW_SELECTION", "analysis_site = '1' Or analysis_site = '2' And Management_Area = 'Kuskokwim'", None)

arcpy.SpatialJoin_analysis("pointsTempLayer", cats, out_pts)
arcpy.management.JoinField(wtds, "outlet_comid", out_pts, "COMID", "Management_Area;Location;location_short;discharge;analysis_site", "NOT_USE_FM", None)



<Result 'W:\\GIS\\AYK_Chinook\\AYK_Chinook.gdb\\wtds_merge'>

This watershed feature class has accurate watersheds for all escapement monitoring locations and the major spawning areas in the Yukon. Since we are attributing environmental information (stream temperature and streamflow) using these watersheds and not the genetic groups, we don't need any of the watersheds provided by Brendan. The Kuskokwim watersheds has some errors anyways, likely because the old HUC12 used to create them were wrong.

We can use this watershed feature class to calculate glacier cover and process precipitation covariates.

In [ ]:
wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                    zone_fields=wtd_cur_fields[0],
                                                                    in_class_features=glac_fc,
                                                                    out_table=wtd_merge_glac_table_path,
                                                                    class_fields='O1Region',
                                                                    out_units="SQUARE_METERS"
                                                                    )

Create watersheds for eight streamflow sites we are using to validate modeled streamflow and
gridded precipitation as surrogates for fall peak flows and scouring of redds.

Note: saved this code in case we want to run more streamflow sites. But could easily just modify point class above. That seems like the best workflow. Start with a good point file with all the necessary attributes, make sure it is in the right merit-basin catchments, and then run everything above.

In [3]:
#from 1_GRFR_validation.rmd script
idList = [81024048, 81024473, 81018427, 81014663, 81014557, 81014712, 81023826, 81019415]

# Find upstream COMIDs and merge to create watersheds

import os
import arcpy
import pandas as pd

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(4326)  #'WGS 1984'
arcpy.env.outputCoordinateSystem = sr

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
streams = r"W:\GIS\MERIT-Basins\riv_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"
cats = r"W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp"

# Get list of index names for cats and add index if not already created
index_names = [i.name for i in arcpy.ListIndexes(cats)]
print(index_names)
if 'comid_index' not in index_names:
    print (f'Creating index for {cats}')
    arcpy.AddIndex_management(cats, "COMID", "comid_index")
else:
    print(f'{cats} Indexed')

#watersheds feature dataset for storing fcs
fdat = os.path.join(ayk_chinook_gdb, 'Watersheds_flowSites')
if not arcpy.Exists(fdat):
    arcpy.management.CreateFeatureDataset(ayk_chinook_gdb, "Watersheds_flowSites", sr)
else:
    print(f'{fdat} exists for AYK Chinook geodatabase')


fields = arcpy.ListFields(streams)
for field in fields:
    print("{0}".format(field.name))

str_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(streams, ("COMID", "up1", "up2", "up3")))
hws_codes = [0]

#idList if doing ALL watersheds.
c = 1
for id in idList:
    print(f"{c}. Starting watershed for: " + str(id))
    rec = [id]
    up_ids = []
    sum_rec = sum(rec)

    while(sum_rec > 0):
        up_ids.append(rec)
        rec = str_df.loc[str_df["COMID"].isin(rec), ("up1", "up2", "up3")]
        rec = pd.concat([rec['up1'], rec['up2'], rec['up3']])
        sum_rec = sum(rec)

    #up_ids is a list with more than numbers, use extend to only keep numeric comids
    newup_ids = []
    for x in up_ids:
        newup_ids.extend(x)

    tempLayer = "catsLyr"
    expression = '"COMID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
    arcpy.MakeFeatureLayer_management(cats, tempLayer)
    arcpy.management.SelectLayerByAttribute(tempLayer, "NEW_SELECTION", expression, None)
    print("Starting dissolve")
    #outdis = "memory/wtd_" + str(round(id))
    outwtd = "Watersheds_flowSites\\flowWtd_" + str(round(id))
    dis = arcpy.Dissolve_management(tempLayer, outwtd)
    #watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
    print("Watershed created at:" + outwtd)
    c=c+1

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')

['COMID']
Creating index for W:\GIS\MERIT-Basins\cat_pfaf_8_MERIT_Hydro_v07_Basins_v01_bugfix1.shp
W:\GIS\AYK_Chinook\AYK_Chinook.gdb\Watersheds_flowSites exists for AYK Chinook geodatabase
FID
Shape
COMID
lengthkm
lengthdir
sinuosity
slope
uparea
order
strmDrop_t
slope_taud
NextDownID
maxup
up1
up2
up3
up4
1. Starting watershed for: 81024048
Starting dissolve
Watershed created at:Watersheds_flowSites\flowWtd_81024048
2. Starting watershed for: 81024473
Starting dissolve
Watershed created at:Watersheds_flowSites\flowWtd_81024473
3. Starting watershed for: 81018427
Starting dissolve
Watershed created at:Watersheds_flowSites\flowWtd_81018427
4. Starting watershed for: 81014663
Starting dissolve
Watershed created at:Watersheds_flowSites\flowWtd_81014663
5. Starting watershed for: 81014557
Starting dissolve
Watershed created at:Watersheds_flowSites\flowWtd_81014557
6. Starting watershed for: 81014712
Starting dissolve
Watershed created at:Watersheds_flowSites\flowWtd_81014712
7. Starting w

In [4]:
# Merge all watersheds into one feature class
import arcpy
arcpy.env.overwriteOutput = True

ayk_chinook_gdb = r"W:\GIS\AYK_Chinook\AYK_Chinook.gdb"
arcpy.env.workspace = ayk_chinook_gdb + "\\Watersheds_flowSites"
wtds = arcpy.ListFeatureClasses()

arcpy.env.workspace = ayk_chinook_gdb
wtdMerge = [ayk_chinook_gdb + "\\Watersheds_flowSites\\" + s for s in wtds]
wtd_output = "flowWtds_merge"
wtd_merge = arcpy.Merge_management(wtdMerge, wtd_output)

print('Process complete')

Process complete
